# 🔗 RSP (Resource Sharing Protocol) — Colab Demo

This notebook demonstrates the core logic of **RSP**, a custom binary protocol for buying/selling virtual machines (VMs) securely.

### ✅ Features

- High-performance binary protocol (MessagePack + struct)
- Packet header format (8 bytes: CMD_ID, FLAGS, LENGTH, RESERVED)
- Secure VM access via SSH (simulated)
- Basic escrow logic with buyer/seller flow

---

## Header
```
RSP_VER   : 1   → protocol version 1
VPN_FLAG  : 1   → tunneled through VPN
CMD_ID    : 1   → REQUEST_VM
FLAGS     : 6   → Encrypted + High Priority
BODY_LEN  : 48  → 0x0030 (big endian)
RESERVED  : 00 00 00 00
```

## 📦 Packet Format

```

\[CMD\_ID:1]\[FLAGS:1]\[BODY\_LEN:2]\[RESERVED:4]\[BODY:...]

````

- `CMD_ID=1` → `REQUEST_VM`
- `FLAGS=6`  → `Encrypted + High Priority`
- `BODY`     → MessagePack dict

---

## 🚀 Run in Colab

```python
!pip install msgpack
````

Then run the notebook cells to:

1. Construct and send RSP packets
2. Parse and print headers
3. Decode and inspect payloads

---

## 💡 Example Payload

```json
{
  "cpu": 2,
  "ram": 4,
  "duration": 60,
  "provider": "qemu"
}
```

---

## 🧠 Next Steps

* Add real SSH validation for VMs
* Integrate escrow logic
* Wrap in OpenVPN + TLS for secure communication
* Signature In-Body
* Session ID
* 🔐 Zero-Knowledge Proof field for spec validation

* 🛠 Command negotiation
* 📊 Usage telemetry *(client asks which VM providers are supported)*

---

*MIT License | Author: Bader Alotaibi*


In [13]:
import msgpack
import struct
import binascii
import json

# Header info
RSP_VER = 1
VPN_FLAG = 0
cmd_id = 1       # REQUEST_VM
flags = 0        # No special flags
#body_len = len(body)
reserved = b'\x00\x00\x00\x00'


# Construct payload
payload = {
  "cpu": 2,
  "ram": 4,
  "duration": 60,
  "provider": "qemu"
}




In [14]:
# Mapping for command IDs
CMD_ID_MAP = {
    1: "REQUEST_VM",
    2: "VM_OFFER",
    3: "ACCEPT_OFFER",
    4: "VM_READY",
    5: "ESCROW_RESULT",
    0xFF: "ERROR"
}

# Mapping for flags (bitwise)
FLAGS_MAP = {
    0: "No flags",
    1: "ACK",
    2: "Encrypted",
    4: "High Priority"
}


In [15]:
body = msgpack.packb(payload)
body_len = len(body)
# Pack header
header = struct.pack(">BBH4s", cmd_id, flags, body_len, reserved)

# Combine header + body
packet = header + body

In [16]:


# ----- Pretty Print -----
print("\n📦 Full RSP Packet (raw bytes):")
print(binascii.hexlify(packet, " ").decode().upper())

print("\n🔍 Parsed RSP Header:")
print(f"  RSP_VER  : {RSP_VER}")
print(f"  VPN_FLAG : {VPN_FLAG}")
print(f"  CMD_ID   : {cmd_id} ({CMD_ID_MAP.get(cmd_id, 'UNKNOWN')})")
print(f"  FLAGS    : {flags} ({FLAGS_MAP.get(flags, 'Unknown or Multiple Flags')})")
print(f"  BODY_LEN : {body_len} bytes")
print(f"  RESERVED : {binascii.hexlify(reserved).decode().upper()}")

print("\n📨 Decoded Payload (MessagePack → JSON):")
print(json.dumps(msgpack.unpackb(body), indent=2))


📦 Full RSP Packet (raw bytes):
01 00 00 23 00 00 00 00 84 A3 63 70 75 02 A3 72 61 6D 04 A8 64 75 72 61 74 69 6F 6E 3C A8 70 72 6F 76 69 64 65 72 A4 71 65 6D 75

🔍 Parsed RSP Header:
  RSP_VER  : 1
  VPN_FLAG : 0
  CMD_ID   : 1 (REQUEST_VM)
  FLAGS    : 0 (No flags)
  BODY_LEN : 35 bytes
  RESERVED : 00000000

📨 Decoded Payload (MessagePack → JSON):
{
  "cpu": 2,
  "ram": 4,
  "duration": 60,
  "provider": "qemu"
}
